<a href="https://colab.research.google.com/github/SeeeRGo/ml-experiments/blob/main/RussianPostHackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import training data set

In [1]:
import torch
from torch import nn
import pandas as pd
from google.colab import auth
from googleapiclient.discovery import build

import io
from googleapiclient.http import MediaIoBaseDownload

In [2]:
auth.authenticate_user()
drive_service = build('drive', 'v3')


In [3]:
request = drive_service.files().get_media(fileId='1Cu_Su0BpJeLzInuF2M5wnsy9Pu9hxXi8')
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}')
train_dataframe = pd.read_csv(downloaded)
train_dataframe.head()

Downloaded file contents are: {}


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629.0,Участок,7503.0,N,0.0,Y,N,87.0,...,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976.0,ММПО,7503.0,N,0.0,N,N,107.0,...,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962.0,Цех,7503.0,N,0.0,Y,N,50.0,...,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0
3,9020937,1019_-1,344964.0,Цех,7503.0,N,0.0,Y,N,416.0,...,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0
4,3082311,1020_-1,629819.0,Участок,7503.0,N,0.0,Y,N,795.0,...,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0


In [4]:
train_dataframe.columns

Index(['id', 'oper_type + oper_attr', 'index_oper', 'type', 'priority',
       'is_privatecategory', 'class', 'is_in_yandex', 'is_return', 'weight',
       'mailtype', 'mailctg', 'mailrank', 'directctg', 'transport_pay',
       'postmark', 'name_mfi', 'weight_mfi', 'price_mfi',
       'dist_qty_oper_login_1', 'total_qty_oper_login_1',
       'total_qty_oper_login_0', 'total_qty_over_index_and_type',
       'total_qty_over_index', 'is_wrong_sndr_name', 'is_wrong_rcpn_name',
       'is_wrong_phone_number', 'is_wrong_address', 'label'],
      dtype='object')

In [5]:
only_numbers = train_dataframe[["weight", "transport_pay"]]
labels = train_dataframe["label"]
X = torch.tensor(only_numbers.values).type(torch.float)
y = torch.tensor(labels.values).type(torch.float)
X.shape, y.shape

(torch.Size([6000000, 2]), torch.Size([6000000]))

### Make a split on training, validation and test data

I think 70/15/15 would be good enough

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.3, shuffle = True, random_state = 42)

# Use the same function above for the validation set
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, 
    test_size=0.5, random_state= 8) # 0.5 x 0.3 = 0.15


print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y val shape: {}".format(y_val.shape))

X_train shape: torch.Size([4200000, 2])
X_test shape: torch.Size([900000, 2])
y_train shape: torch.Size([4200000])
y_test shape: torch.Size([900000])
X_val shape: torch.Size([900000, 2])
y val shape: torch.Size([900000])


In [8]:
# Building trainig loop
from sklearn import metrics
model = nn.Sequential(nn.Linear(in_features=2, out_features=32),
                      nn.ReLU(),
                      nn.Linear(in_features=32, out_features=32),
                      nn.ReLU(),
                      nn.Linear(in_features=32, out_features=1))

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=0.05)
epochs = 100

model.train()
for epoch in range(epochs):
  y_logits = model(X_train).squeeze()

  y_preds = torch.sigmoid(y_logits)

  y_preds = torch.round(y_preds)
  # print(f"Y_PREDS {y_preds[:5]}")

  loss = loss_fn(y_logits, y_train)
  auc_roc = metrics.roc_auc_score(y_train.detach().numpy(), y_preds.detach().numpy())

  # accuracy = accuracy_fn(y_true=y_train, y_pred=y_preds)

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  model.eval()
  with torch.inference_mode():
    # 1. Forward pass
    # test_logits = model(X_test).squeeze()
    # test_pred = torch.round(torch.sigmoid(test_logits))

    # # test_pred = torch.sigmoid(test_pred)

    # # test_pred = torch.round(test_pred)

    # # 2. Calculate the loss
    # test_loss = loss_fn(test_pred.squeeze(), y_test)
    # test_accuracy = accuracy_fn(y_true=y_test, y_pred=test_pred)
    y_val_logits = model(X_val).squeeze()

    y_val_preds = torch.round(torch.sigmoid(y_val_logits))

    val_loss = loss_fn(y_val_logits, y_val)

    val_auc_roc = metrics.roc_auc_score(y_val.detach().numpy(), y_val_preds.detach().numpy())

    # epochs_count.append(epoch)
    # loss_values.append(loss)
    # test_loss_values.append(test_loss)
    # if epoch % 15 == 0:
    #   print(f"Epoch: {epoch} | Loss: {loss} | Test loss: {test_loss} | Accuracy: {accuracy} | Test Accuracy: {test_accuracy}")
    if epoch % 10 == 0:
      print(f"Epoch: {epoch} | Loss: {loss} | AUC: {auc_roc} | Validation Loss: {val_loss} | Validation AUC: {val_auc_roc}")




Epoch: 0 | Loss: 5.723521709442139 | AUC: 0.5 | Validation Loss: 87.64808654785156 | Validation AUC: 0.5
Epoch: 10 | Loss: 0.2241925448179245 | AUC: 0.5 | Validation Loss: 0.22081729769706726 | Validation AUC: 0.5
Epoch: 20 | Loss: 0.1917504221200943 | AUC: 0.5 | Validation Loss: 0.18749722838401794 | Validation AUC: 0.5
Epoch: 30 | Loss: 0.16921499371528625 | AUC: 0.5 | Validation Loss: 0.16794846951961517 | Validation AUC: 0.5
Epoch: 40 | Loss: 0.15777277946472168 | AUC: 0.5 | Validation Loss: 0.15742290019989014 | Validation AUC: 0.5
Epoch: 50 | Loss: 0.1512572318315506 | AUC: 0.5 | Validation Loss: 0.15119026601314545 | Validation AUC: 0.5
Epoch: 60 | Loss: 0.14657479524612427 | AUC: 0.5 | Validation Loss: 0.14664632081985474 | Validation AUC: 0.5
Epoch: 70 | Loss: 0.14300279319286346 | AUC: 0.5 | Validation Loss: 0.14317788183689117 | Validation AUC: 0.5
Epoch: 80 | Loss: 0.13993880152702332 | AUC: 0.5 | Validation Loss: 0.14010067284107208 | Validation AUC: 0.5
Epoch: 90 | Loss: 

In [ ]:
model.state_dict()